<a href="https://colab.research.google.com/github/RafalDoroz/ai/blob/main/podpisy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import re
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay

# Program do rozpoznawania podpisów
# Celem tego programu jest analiza i klasyfikacja podpisów użytkowników na podstawie danych w formacie TXT.

# Krok 1: Przygotowanie zbioru danych uczących
print("Krok 1: Przygotowanie zbioru danych uczących")
sig_Features = []
user_ID = []

for filename in os.listdir("signatures"):
    with open(os.path.join("signatures", filename), 'r') as f:
        sig = pd.read_csv(f)
        print("Przetwarzanie pliku:", filename)

        # Przetwarzanie danych podpisu
        sig = sig.drop(sig.index[[0]])
        sig = sig.iloc[:, 0].str.split(' ', expand=True)
        sig = sig.astype(str).astype(int)
        sig.columns = ['X', 'Y', 'T', 'S', 'Az', 'Al', 'P']

        # Ekstrakcja ID użytkownika z nazwy pliku
        filename_parts = re.split('S|U|\.', filename)
        user_ID.append(filename_parts[1])

        # Obliczenia różnic i prędkości
        sig['Y_diff'] = sig['Y'].diff()
        sig.iloc[0, -1] = sig.iloc[1, -1]
        sig['T_diff'] = sig['T'].diff()
        sig.iloc[0, -1] = sig.iloc[1, -1]
        sig['Y_vel'] = sig['Y_diff'] / sig['T_diff']
        sig.drop(sig[sig['T_diff'] == 0].index, inplace=True)

        # Wartości maksymalne i minimalne dla Y_vel
        maxValues = sig['Y_vel'].max()
        minValues = sig['Y_vel'].min()

        # Feature extraction: liczba punktów, max i min wartości Y_vel
        sig_Features.append([sig.shape[0], minValues, maxValues])

# Krok 2: Tworzenie modelu Decision Tree
print("\nKrok 2: Tworzenie modelu Decision Tree")
clf = DecisionTreeClassifier()
clf = clf.fit(sig_Features, user_ID)

# Krok 3: Testowanie modelu
print("\nKrok 3: Testowanie modelu")
test_features = []
test_user_ID = []

for test_filename in os.listdir("signatures"):
    with open(os.path.join("signatures", test_filename), 'r') as f:
        test_sig = pd.read_csv(f)
        test_sig = test_sig.iloc[:, 0].str.split(' ', expand=True)
        test_sig = test_sig.drop(test_sig.index[[0]])
        test_sig = test_sig.astype(str).astype(int)
        test_sig.columns = ['X', 'Y', 'T', 'S', 'Az', 'Al', 'P']

        # Obliczenia różnic i prędkości w danych testowych
        test_sig['Y_diff'] = test_sig['Y'].diff()
        test_sig.iloc[0, -1] = test_sig.iloc[1, -1]
        test_sig['T_diff'] = test_sig['T'].diff()
        test_sig.iloc[0, -1] = test_sig.iloc[1, -1]
        test_sig['Y_vel'] = test_sig['Y_diff'] / test_sig['T_diff']
        test_sig.drop(test_sig[test_sig['T_diff'] == 0].index, inplace=True)

        # Ekstrakcja cech z danych testowych
        test_maxValues = test_sig['Y_vel'].max()
        test_minValues = test_sig['Y_vel'].min()
        test_features.append([test_sig.shape[0], test_minValues, test_maxValues])

        # Ekstrakcja ID użytkownika z nazwy pliku
        filename_parts = re.split('S|U|\.', test_filename)
        test_user_ID.append(filename_parts[1])

# Predykcje na podstawie danych testowych
predictions = clf.predict(test_features)

# Obliczanie skuteczności
accuracy = accuracy_score(test_user_ID, predictions)
print("\nSkuteczność modelu: {:.2f}%".format(accuracy * 100))

# Wyświetlanie szczegółowych wyników
for i, pred in enumerate(predictions):
    print(f"Plik testowy: {os.listdir('signatures')[i]}, Przewidywany użytkownik: {pred}, Rzeczywisty użytkownik: {test_user_ID[i]}")

# Obliczanie macierzy pomyłek
conf_matrix = confusion_matrix(test_user_ID, predictions, labels=np.unique(user_ID))
print("\nMacierz pomyłek:")
print(conf_matrix)

# Wyświetlanie macierzy pomyłek
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=np.unique(user_ID))
disp.plot(cmap='viridis')
plt.title("Macierz pomyłek dla klasyfikatora")
plt.show()

# Wyświetlanie trzech pierwszych wektorów cech
print("\nTrzy pierwsze wektory cech:")
for i in range(min(5, len(sig_Features))):
    print(f"Wektor {i + 1}: Liczba punktów: {sig_Features[i][0]}, Minimalna wartość Y_vel: {sig_Features[i][1]}, Maksymalna wartość Y_vel: {sig_Features[i][2]}")


Krok 1: Przygotowanie zbioru danych uczących


FileNotFoundError: [Errno 2] No such file or directory: 'signatures'